In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup, Comment
import pandas as pd
from datetime import date
import numpy as np
import requests
import re

pd.set_option('mode.chained_assignment', None)

In [6]:
salaries = pd.read_csv('temp_csvs/players_1985to2018.csv', sep = ',')
IDs = salaries[['_id','name']]

IDs['url'] = 'https://www.basketball-reference.com/players/' + IDs._id.str[0] + '/' + IDs._id + '.html'

IDs.head()

,_id,name,url
0,abdelal01,Alaa Abdelnaby,https://www.basketball-reference.com/players/a...
1,abdulza01,Zaid Abdul-Aziz,https://www.basketball-reference.com/players/a...
2,abdulka01,Kareem Abdul-Jabbar,https://www.basketball-reference.com/players/a...
3,abdulma02,Mahmoud Abdul-Rauf,https://www.basketball-reference.com/players/a...
4,abdulta01,Tariq Abdul-Wahad,https://www.basketball-reference.com/players/a...


In [ ]:
dfs = []

for i in range(len(IDs)):

    url = IDs.iloc[i, 2]
    name = IDs.iloc[i, 1]
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'html.parser')
    placeholder = soup.select_one('#all_shooting .placeholder')
    
    if placeholder is not None: # some of the urls do not contain shooting stats tables
        
        comment = next(elem for elem in placeholder.next_siblings if isinstance(elem, Comment))
        table_soup = BeautifulSoup(comment, 'html.parser')

        headers = [th.getText() for th in table_soup.findAll('th')]
        stop1 = headers.index('Career')  # there is a variable number of seasons to consider; 'Career'
                                        # is the last 'th' after each season before format changes
        seasons = headers[40:stop1] # seasons, which belong in rows but are also classified as 'th'
        headers = headers[12:40] # headers we actually want
        
        data = [] # data for each header

        rows = table_soup.find_all('tr')
        for row in rows:
            cells = row.find_all('td')
            if cells:
                data.append([cell.text for cell in cells])

        stop2 = [a for (a, b) in enumerate(data) if b[0]==''][0]
        data = data[:stop2] # leave out career averages/totals

        shooting_df = pd.DataFrame(data, columns = headers)
        shooting_df['Year'] = seasons
        shooting_df.Year = pd.to_numeric(shooting_df.Year.str[:4]) # converting '2009-2010' string to '2009' integer
        shooting_df['Full Name'] = name

        df = shooting_df.iloc[:,[1,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29]]

        df.columns = ['Tm','FG%','Dist.','2P_A','0-3_A',
                      '3-10_A','10-16_A','16-3pt_A','3P_A','2P%','0-3%','3-10%','10-16%','16-3pt%',
                      '3P%',"Astd_2P",'Dunk%','Dunks_Md',"Astd_3P",'%Corner3_A','Corner_3P%','Heaves_A', 
                      'Heaves_Md','Year','Full Name']

        df = df[['Full Name','Year','Tm','FG%','Dist.','2P_A','0-3_A','3-10_A', '10-16_A', '16-3pt_A', 
                 '3P_A', '2P%', '0-3%', '3-10%', '10-16%', '16-3pt%','3P%', "Astd_2P", 'Dunk%', 
                 'Dunks_Md', "Astd_3P", '%Corner3_A', 'Corner_3P%', 'Heaves_A', 'Heaves_Md']]

        dfs.append(df)

shot_df = pd.concat(dfs)

In [ ]:
shot_df = shot_df.replace('',np.nan)

In [ ]:
shot_df.iloc[:,range(3,25)] = shot_df.iloc[:,range(3,25)].astype('float')

In [ ]:
file_loc = 'temp_csvs/shooting.csv'

shot_df.to_csv(file_loc, sep = '\t')